In [1]:
import pandas  as pd
import datetime
import time

In [2]:
train_df = pd.read_csv('train.csv')

In [3]:
train_df.shape

(878049, 9)

In [4]:
train_df = train_df[abs(train_df['Y'])< 90]

In [5]:
train_df.shape

(877982, 9)

In [6]:
train_df['Dates'] = pd.to_datetime(train_df['Dates'])

In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
sc = StandardScaler()

In [9]:
sc.fit(train_df[['X','Y']])

StandardScaler(copy=True, with_mean=True, with_std=True)

In [10]:
train_df[['X','Y']] = sc.transform(train_df[['X','Y']])

In [11]:
train_df.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-0.123732,0.313018
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-0.123732,0.313018
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-0.063274,1.381346
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-0.167381,1.400312
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-0.631787,0.186493


In [12]:
#locations = pd.get_dummies(train_df['Address'])

In [13]:
#locations.shape

In [14]:
hour_of_day = (pd.to_datetime(train_df['Dates'])).dt.hour

In [15]:
month = pd.get_dummies((pd.to_datetime(train_df['Dates'])).dt.month, drop_first = True)

In [16]:
year = pd.get_dummies((pd.to_datetime(train_df['Dates'])).dt.year, drop_first = True)

In [17]:
day_of_week = pd.get_dummies(train_df['DayOfWeek'], drop_first = True)

In [18]:
police_district = pd.get_dummies(train_df['PdDistrict'] , drop_first = True)

In [19]:
X = pd.DataFrame(columns = ['hour','long', 'lat','district'])

In [20]:
X['hour'] = hour_of_day
X['long'] = train_df['X']
X['lat'] = train_df['Y']
X['district'] = police_district

In [21]:
X = pd.concat([X , day_of_week, month, year], axis = 1)

In [22]:
X.head()

,hour,long,lat,district,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,23,-0.123732,0.313018,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,23,-0.123732,0.313018,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,23,-0.063274,1.381346,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,23,-0.167381,1.400312,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,23,-0.631787,0.186493,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [23]:
X.shape

(877982, 33)

In [24]:
y = train_df['Category']

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
clf =  RandomForestClassifier(n_estimators = 100,max_depth = 20)

In [27]:
clf.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=20, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [28]:
from sklearn.metrics import accuracy_score

In [29]:
train_pred = clf.predict(X)

In [30]:
train_pred

array(['OTHER OFFENSES', 'OTHER OFFENSES', 'LARCENY/THEFT', ...,
       'LARCENY/THEFT', 'LARCENY/THEFT', 'OTHER OFFENSES'], dtype=object)

In [31]:
print('Accuracy score - ', accuracy_score(y,train_pred))

Accuracy score -  0.495895132246447


In [32]:
comparison_df = pd.DataFrame(columns = ['Actual' , 'Predicted'])

In [33]:
comparison_df['Actual'] = y
comparison_df['Predicted'] = train_pred

In [34]:
comparison_df.head(25)

,Actual,Predicted
0,WARRANTS,OTHER OFFENSES
1,OTHER OFFENSES,OTHER OFFENSES
2,OTHER OFFENSES,LARCENY/THEFT
3,LARCENY/THEFT,LARCENY/THEFT
4,LARCENY/THEFT,LARCENY/THEFT
5,LARCENY/THEFT,LARCENY/THEFT
6,VEHICLE THEFT,VEHICLE THEFT
7,VEHICLE THEFT,VEHICLE THEFT
8,LARCENY/THEFT,LARCENY/THEFT
9,LARCENY/THEFT,LARCENY/THEFT
